In [2]:
 pip install catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 8.1 MB/s eta 0:00:00


In [4]:
#Importing the required packages

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.impute import KNNImputer
from catboost import CatBoostClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score

#Reading the dataset
data = pd.read_csv('/content/train.csv')

#Displaying the first 5 rows of the dataset
data.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


In [5]:
#Getting the datatypes of the dataset
data.dtypes

,0
PassengerId,object
HomePlanet,object
CryoSleep,object
Cabin,object
Destination,object
Age,float64
VIP,object
RoomService,float64
FoodCourt,float64
ShoppingMall,float64


In [12]:

# Assuming data_updated is your DataFrame with all necessary columns
data_updated = data_updated.dropna()  # Optional: drop rows with NaN values if needed

# Define the features (X) and the target variable (y)
X = data_updated[['PassengerId', 'HomePlanet', 'CryoSleep', 'Deck', 'Num', 'Side', 'Destination', 'Age', 'VIP', 'Total Expenses']]
y = data_updated['Transported']

# Convert categorical features to 'category' type using .loc
categorical_cols = ['HomePlanet', 'CryoSleep', 'Deck', 'Side', 'Destination', 'VIP']
for col in categorical_cols:
    X.loc[:, col] = X[col].astype('category')  # Use .loc to avoid SettingWithCopyWarning

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [13]:

data_TotalExpenses = data[['RoomService', 'FoodCourt','ShoppingMall','Spa']]

# Calculate sum of each row and create 'sum' column
data_TotalExpenses['Total Expenses'] = data_TotalExpenses.sum(axis=1)

# Create 'Total Expenses' column in the original DataFrame
data['Total Expenses'] = data_TotalExpenses['Total Expenses']

data_updated = data[['PassengerId', 'HomePlanet', 'CryoSleep', 'Cabin', 'Destination', 'Age', 'VIP', 'Transported', 'Total Expenses']]

# Splitting the 'Cabin' column
data_updated[['Deck','Num', 'Side']] = data_updated['Cabin'].str.split('/', expand=True)

# Selecting the desired columns for the final data_updated DataFrame
data_updated = data_updated[['PassengerId', 'HomePlanet', 'CryoSleep', 'Deck', 'Num', 'Side', 'Destination', 'Age', 'VIP', 'Total Expenses','Transported']]
data_updated

<ipython-input-13-d65f23f33042>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_TotalExpenses['Total Expenses'] = data_TotalExpenses.sum(axis=1)
<ipython-input-13-d65f23f33042>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_updated[['Deck','Num', 'Side']] = data_updated['Cabin'].str.split('/', expand=True)
<ipython-input-13-d65f23f33042>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See t

,PassengerId,HomePlanet,CryoSleep,Deck,Num,Side,Destination,Age,VIP,Total Expenses,Transported
0,0001_01,Europa,False,B,0,P,TRAPPIST-1e,39.0,False,0.0,False
1,0002_01,Earth,False,F,0,S,TRAPPIST-1e,24.0,False,692.0,True
2,0003_01,Europa,False,A,0,S,TRAPPIST-1e,58.0,True,10334.0,False
3,0003_02,Europa,False,A,0,S,TRAPPIST-1e,33.0,False,4983.0,False
4,0004_01,Earth,False,F,1,S,TRAPPIST-1e,16.0,False,1089.0,True
...,...,...,...,...,...,...,...,...,...,...,...
8688,9276_01,Europa,False,A,98,P,55 Cancri e,41.0,True,8462.0,False
8689,9278_01,Earth,True,G,1499,S,PSO J318.5-22,18.0,False,0.0,False
8690,9279_01,Earth,False,G,1500,S,TRAPPIST-1e,26.0,False,1873.0,True
8691,9280_01,Europa,False,E,608,S,55 Cancri e,32.0,False,1402.0,False


In [14]:
#print the number of missing values in each column
print(data_updated.isnull().sum())

PassengerId         0
HomePlanet        201
CryoSleep         217
Deck              199
Num               199
Side              199
Destination       182
Age               179
VIP               203
Total Expenses      0
Transported         0
dtype: int64


In [26]:
# Inspect the 'Num' column for problematic values
print("Unique values in 'Num' before processing:", X_train['Num'].unique())

# Check and add 'missing' category to the 'Num' column
if not isinstance(X_train['Num'].dtype, pd.CategoricalDtype):
    # Convert to category if not already
    X_train['Num'] = X_train['Num'].astype('category')
    X_test['Num'] = X_test['Num'].astype('category')

# Add 'missing' category if it does not exist
X_train['Num'] = X_train['Num'].cat.add_categories('missing')
X_test['Num'] = X_test['Num'].cat.add_categories('missing')

# Now fill NaN values with 'missing'
X_train['Num'].fillna('missing', inplace=True)
X_test['Num'].fillna('missing', inplace=True)

# Ensure all other categorical columns are treated as categorical
categorical_cols = ['HomePlanet', 'CryoSleep', 'Deck', 'Side', 'Destination', 'VIP', 'Num']
for col in categorical_cols:
    if not isinstance(X_train[col].dtype, pd.CategoricalDtype):
        X_train[col] = X_train[col].astype('category')
    if not isinstance(X_test[col].dtype, pd.CategoricalDtype):
        X_test[col] = X_test[col].astype('category')

# Debugging: Check the types of the categorical columns
print(X_train[categorical_cols].dtypes)

# Fit your CatBoost model
model = CatBoostClassifier()
try:
    model.fit(X_train, y_train, cat_features=categorical_cols)
except Exception as e:
    print("Error during model fitting:", e)

# Make predictions on the test set
predictions = model.predict(X_test)

# Optionally, evaluate the model
accuracy = accuracy_score(y_test, predictions)
print(f"Accuracy: {accuracy:.2f}")


Unique values in 'Num' before processing: ['18', '1156', '1266', '1205', '564', ..., '519', '430', '622', '1770', '1031']
Length: 1693
Categories (1694, object): ['0', '1', '10', '100', ..., '997', '998', '999', 'missing']


ValueError: new categories must not include old categories: {'missing'}

In [27]:


# Prepare features and target variable for CatBoost
X = data_updated.drop(columns=['PassengerId', 'Transported'])
y = data_updated['Transported'].astype(int)  # Convert to integer (0 or 1)

# Define categorical columns for CatBoost
categorical_cols = ['CryoSleep', 'Deck', 'Side', 'Destination', 'VIP']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Initialize CatBoostClassifier model
model = CatBoostClassifier(iterations=500, learning_rate=0.05, depth=8, l2_leaf_reg=5, verbose=100)

# Fit the model, explicitly specifying categorical features
model.fit(X_train, y_train, cat_features=categorical_cols)

# Make predictions on the test data
y_pred = model.predict(X_test)
y_pred_proba = model.predict_proba(X_test)[:, 1]  # Get probabilities for the positive class

# Calculate evaluation metrics
accuracy = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_pred_proba)

# Print the results
print(f"Accuracy: {accuracy:.4f}")
print(f"F1 Score: {f1:.4f}")
print(f"AUC-ROC: {roc_auc:.4f}")


CatBoostError: Bad value for num_feature[non_default_doc_idx=0,feature_idx=0]="Earth": Cannot convert 'b'Earth'' to float